# Assignment Combine Data Solution

## About the data

The data is generated by Synthea's COVID-19 module. The data was constructed using three peer-reviewed publications published in the early stages of the global pandemic, when less was known, along with emerging resources, data, publications, and clinical knowledge. The simulation outputs synthetic Electronic Health Records (EHR), including the daily consumption of Personal Protective Equipment (PPE) and other medical devices and supplies. For this assignment the `conditions`, `patients`, `observations`, `careplans` and `encounters` table will be used. The Data is stored in separate tables to avoid redundancy, with as a concequence that tables need to be combined and reorganized in dataframes for analysing purpose.

Source: Walonoski J, Klaus S, Granger E, Hall D, Gregorowicz A, Neyarapally G, Watson A, Eastman J. Synthea™ Novel coronavirus (COVID-19) model and synthetic data set. Intelligence-Based Medicine. 2020 Nov;1:100007. https://doi.org/10.1016/j.ibmed.2020.100007

Please <a href = "https://storage.googleapis.com/synthea-public/10k_synthea_covid19_csv.zip">download</a> the data

#### Covid Patients
Patients are considered Covid patients if they are identified with `CODE` `840539006`


#### Survivors
Patients that had covid and where tested negative after isolation have tested code `94531-1`,  SARS-CoV-2 RNA Pnl Resp NAA+probe (covid-sars test) + a value of `Not detected (qualifier value)`. These patients are considered to be survived covid patients. 

#### Non-Survivors
Patients that did not survived Covid have a `DEATHDATE` which is not null. 


#### Lab values  COVID-19 patients

Patients are monitored for blood and heart conditions once they are admitted in Hospital or under treatment. The lab values of interest are as follow: 

- `48065-7`  Fibrin D-dimer FEU [Mass/volume] in Platelet poor plasma
- `26881-3`   Interleukin 6 [Mass/volume] in Serum or Plasma
- `2276-4` Ferritin [Mass/volume] in Serum or Plasma
- `89579-7` Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method
- `731-0` Lymphocytes [#/volume] in Blood by Automated count
- `14804-9` Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction


---

In [1]:
#All imports
import pandas as pd
import numpy as np
from bokeh.models.widgets import Panel, Tabs
from bokeh.models import ColumnDataSource, HoverTool, Circle
from bokeh.plotting import figure, show #, output_file
from bokeh.layouts import gridplot
from bokeh.plotting import ColumnDataSource
from collections import OrderedDict
from bokeh.io import output_notebook
from bokeh.tile_providers import get_provider, ESRI_IMAGERY, CARTODBPOSITRON
output_notebook()

Loading BokehJS ...

<a name='0'></a>
## Part 1: Load the data (20 pt)

Instructions: Load the data of the following files

- conditions.csv
- patients.csv
- observations.csv
- careplans.csv
- encounters.csv

Get yourself familiar with the data. Create some meaningful overviews. Answer the following questions

1. How many patients are there
2. How many covid-patients are there
3. How many patients do have a 'Hospital admission for isolation' encounter
    
<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
    <ul><li>use a unique dataframe for each file, use a meaningful name</li>
    <li>pandas.read_csv() method can be used to read a csv file</li>
    <li>pandas.DataFrame.head() method is often used to inspect the dataframe</li>
    <li>.unique() returns a list of unique values of a column</li>
</ul>
</details>

<a name='ex-11'></a>
### 1.1 Code your solution

In [2]:
#YOUR CODE HERE
conditions = pd.read_csv('data/conditions.csv')
patients = pd.read_csv('data/patients.csv')
observations = pd.read_csv('data/observations.csv')
care_plans = pd.read_csv('data/careplans.csv')
encounters = pd.read_csv('data/encounters.csv')

In [3]:
# - How many patients are there
num_pat = len(set(patients['Id']))
# - How many covid-patients are there
df_cov = conditions[conditions['CODE'] == 840539006].copy() #make a different df for patients with covid
num_cov = len(set(df_cov['PATIENT']))
# - How many patients do have a 'Hospital admission for isolation' encounter
patients_died = patients[~patients['DEATHDATE'].isnull()].copy()
num_died = len(set(patients_died['Id']))
admitted_enco = encounters[encounters['DESCRIPTION'] == 'Hospital admission for isolation (procedure)'].copy()
num_admitted = len(set(admitted_enco['Id']))



### 1.2 Test your solution
The following function needs to be called. You can use this as a test. There are however more meaningful overviews 
you can create. 

In [4]:
def part1(num_pat, num_cov, num_admitted, num_died):
    print(f'There are {num_pat} patients in total')
    print(f'There are {num_cov} covid patients')
    print(f'There are {num_admitted} admitted patients')
    print(f'{num_died} patients died')
    
part1(num_pat, num_cov, num_admitted, num_died)


There are 12352 patients in total
There are 8820 covid patients
There are 1867 admitted patients
2352 patients died


### Expected outcome

---

<a name='1'></a>
## Part 2: Data Wrangling: set up the dataframe (30 pt)

In this part we are going to combine data to create a dataframe with values of interest for the lab values analysis. 

We would like a dataframe containing the following information per record (only Covid patients!!!)

- `PATIENT` - the ID of the covid patient
- `days` - the number of days the patient is under observation
- `CODE-Y` - the code of the observation  
- `VALUE` - the lab value of the observation

where only the following observation codes needs to be selected:

- `48065-7`  Fibrin D-dimer FEU [Mass/volume] in Platelet poor plasma
- `26881-3`   Interleukin 6 [Mass/volume] in Serum or Plasma
- `2276-4` Ferritin [Mass/volume] in Serum or Plasma
- `89579-7` Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method
- `731-0` Lymphocytes [#/volume] in Blood by Automated count
- `14804-9` Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction

The days information is not primarely available and needs to be calculated by substracting observation DATE - START. 

An example of such a dataframe is given below:

In [5]:
#Possible approach:

#Select all the patients with covid from the conditions table
#Combine conditions table (only covid patients) with the patient table into a covid_patient table
#select the only the relevant lab observations from the observations table into a lab_obs table
#merge the covid_patient table with the lab_obs table into a covid_patients_obs table
#clean the covid_patients_obs table (rename columns, select only relevant columns, sort, typecast, add days column)

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
    <ul><li>you can use pandas.DataFrame.merge() to merge dataframes</li>
    <li>df = df[(df.CODE == condition1 | df.CODE == condition1 )] selects rows with CODE of 2 conditional values</li>
    <li>df.DATE - df.START return days if DATE and START are datetime format</li>
    <li>pd.to_datetime() can be used to typecast to datetime</li>
</ul>
</details>

<a name='ex-21'></a>
### 2.1 Code your solution

In [6]:
#Combine conditions table (only covid patients) with the patient table into a covid_patient table
covid_patient = pd.merge(df_cov, patients, left_on=  'PATIENT', right_on= 'Id')

In [7]:
#select the only the relevant lab observations from the observations table into a lab_obs table
lab_obs = observations[observations['CODE'].isin(['48065-7','26881-3', '2276-4', '89579-7', '731-0', '14804-9'])]

In [8]:
#merge the covid_patient table with the lab_obs table into a covid_patients_obs table
covid_patients_obs = pd.merge(covid_patient, lab_obs, on = 'PATIENT')
covid_patients_obs.head()

,START,STOP,PATIENT,ENCOUNTER_x,CODE_x,DESCRIPTION_x,Id,BIRTHDATE,DEATHDATE,SSN,...,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,DATE,ENCOUNTER_y,CODE_y,DESCRIPTION_y,VALUE,UNITS,TYPE
0,2020-02-19,2020-02-28,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,e3143bce-4a59-40aa-a198-7a9e54077fd8,840539006,COVID-19,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,2019-06-12,NaN,999-81-4349,...,-72.722648,22940.0,893.28,2020-02-19,e97e8d37-7497-4c13-98fd-a4a45655c0bb,731-0,Lymphocytes [#/volume] in Blood by Automated c...,1.1,10*3/uL,numeric
1,2020-02-19,2020-02-28,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,e3143bce-4a59-40aa-a198-7a9e54077fd8,840539006,COVID-19,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,2019-06-12,NaN,999-81-4349,...,-72.722648,22940.0,893.28,2020-02-19,e97e8d37-7497-4c13-98fd-a4a45655c0bb,48065-7,Fibrin D-dimer FEU [Mass/volume] in Platelet p...,0.4,ug/mL,numeric
2,2020-02-19,2020-02-28,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,e3143bce-4a59-40aa-a198-7a9e54077fd8,840539006,COVID-19,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,2019-06-12,NaN,999-81-4349,...,-72.722648,22940.0,893.28,2020-02-19,e97e8d37-7497-4c13-98fd-a4a45655c0bb,2276-4,Ferritin [Mass/volume] in Serum or Plasma,332.4,ug/L,numeric
3,2020-02-19,2020-02-28,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,e3143bce-4a59-40aa-a198-7a9e54077fd8,840539006,COVID-19,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,2019-06-12,NaN,999-81-4349,...,-72.722648,22940.0,893.28,2020-02-19,e97e8d37-7497-4c13-98fd-a4a45655c0bb,89579-7,Troponin I.cardiac [Mass/volume] in Serum or P...,2.3,pg/mL,numeric
4,2020-02-19,2020-02-28,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,e3143bce-4a59-40aa-a198-7a9e54077fd8,840539006,COVID-19,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,2019-06-12,NaN,999-81-4349,...,-72.722648,22940.0,893.28,2020-02-19,e97e8d37-7497-4c13-98fd-a4a45655c0bb,14804-9,Lactate dehydrogenase [Enzymatic activity/volu...,223.9,U/L,numeric


In [9]:
#clean the covid_patients_obs table (rename columns, select only relevant columns, sort, typecast, add days column)
#to_datetime
covid_patients_obs[['DATE','START']] = covid_patients_obs[['DATE','START']].apply(pd.to_datetime)
#Calculate days between DATE and START
covid_patients_obs['lab_days'] = covid_patients_obs['DATE'] - covid_patients_obs['START']
covid_patients_obs['days'] = covid_patients_obs.lab_days / np.timedelta64(1, 'D')
# covid_patients_obs['days'] = covid_patients_obs['lab_days'].dt.days
#Select columns
covid_patients_obs_clean = covid_patients_obs[['PATIENT', 'days', 'CODE_y', 'VALUE', 'UNITS']]
#rename column
covid_patients_obs_clean = covid_patients_obs_clean.rename(columns={'CODE_y': 'CODE-Y'})
#Set index
covid_patients_obs_clean = covid_patients_obs_clean.set_index('PATIENT')
# make float of objects
covid_patients_obs_clean['VALUE'] = covid_patients_obs_clean['VALUE'].astype(float)
covid_patients_obs_clean


,days,CODE-Y,VALUE,UNITS
PATIENT,,,,
f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0.0,731-0,1.1,10*3/uL
f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0.0,48065-7,0.4,ug/mL
f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0.0,2276-4,332.4,ug/L
f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0.0,89579-7,2.3,pg/mL
f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0.0,14804-9,223.9,U/L
...,...,...,...,...
c9699449-7a8b-400a-8e86-fab6aa7134cb,8.0,731-0,0.9,10*3/uL
c9699449-7a8b-400a-8e86-fab6aa7134cb,8.0,48065-7,0.5,ug/mL
c9699449-7a8b-400a-8e86-fab6aa7134cb,8.0,2276-4,525.2,ug/L


---

<a name='2'></a>
## Part 3: Data Wrangling, split into survived and not survived (10 pt)

Now we have the required data we would like to split the data into survived and not survived. First we fetch all the ids of the survived and deceased patients. We can use these ids to select the records of the survived patients and the patients that did not survived

Your job is to split the data into survived and not survived records. There are multiple ways to do this. One way is the  `.isin()` method

In [10]:
#the following code is given, RUN THIS CELL
#get survived and deceased ids
completed_isolation_patients = care_plans[(care_plans.CODE == 736376001) & (care_plans.STOP.notna()) \
                                          & (care_plans.REASONCODE == 840539006)].PATIENT
negative_covid_patient_ids = observations[(observations.CODE == '94531-1') \
                                          & (observations.VALUE == 'Not detected (qualifier value)')].PATIENT.unique()
survivor_ids = np.union1d(completed_isolation_patients, negative_covid_patient_ids)
deceased_ids = patients[patients.DEATHDATE.notna()].Id

<a name='ex-31'></a>
### 3.1 Code your solution

In [11]:
#Splot the data into survived and not survived dataframes
survivor_df = covid_patients_obs_clean[covid_patients_obs_clean.index.isin(survivor_ids)]
deceased_df = covid_patients_obs_clean[covid_patients_obs_clean.index.isin(deceased_ids)]
survivor_df.head()

,days,CODE-Y,VALUE,UNITS
PATIENT,,,,
f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0.0,731-0,1.1,10*3/uL
f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0.0,48065-7,0.4,ug/mL
f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0.0,2276-4,332.4,ug/L
f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0.0,89579-7,2.3,pg/mL
f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0.0,14804-9,223.9,U/L


### 3.2 Test your solution

In [12]:
def test3(survived, died):
    print(f'patients records survived: {survived}, patients records deceased {died}')
#call the test3
test3(len(survivor_df), len(deceased_df))

patients records survived: 57303, patients records deceased 16793


#### Expected outcome

---